In [1]:
import os, sys
sys.path.append(os.getcwd()+"/../..")
sys.path.append(os.getcwd()+"/..")
import pandas as pd
from bit_flip_experiments import is_target_qs, backends_w_embs
import seaborn as sns
from utils import load_algorithms_file, Precision, FAKE_HANOI
from game import serialize_algorithms, dump_algorithms
from noise import get_ibm_noise_model
import matplotlib.pyplot as plt

In [2]:
experiments_dir = "../exp_out/"

In [3]:
def find_all_different_programs(outpath, horizon):
    diff_programs = []
    def insert_backend(backend, algorithm):
        found = False
        for i in range(len(diff_programs)):
            if algorithm == diff_programs[i][0]:
                found = True
                diff_programs[i][1].add(backend)
        return found
    for (backend, embeddings) in backends_w_embs:
        for (index, embedding) in enumerate(embeddings):
            algorithms = load_algorithms_file(f"exp_out.bf_{backend}_graphs{index}.bf_{horizon}")
            for algorithm in algorithms:
                if not insert_backend((backend, index), algorithm):
                    p = (backend, index)
                    s = set()
                    s.add(p)
                    diff_programs.append((algorithm, s))

    # serialize_algorithms([x for (x, _) in diff_programs], outpath)
                
    return diff_programs
    

In [4]:
res4 = find_all_different_programs("./exp_out/diff4.py", 4)

In [5]:
res5 = find_all_different_programs("./exp_out/diff5.py", 5)

In [6]:
res6 = find_all_different_programs("./exp_out/diff6.py",6)

In [7]:
len(res5)

7

In [8]:
groups5 = dict()
# def insert_group(index, index2):
#     if index == index2:
#         groups5[index] = 
for (index, data) in enumerate(res6):
    for (index2, data2) in enumerate(res6):
        if index != index2:
            if data[1].issubset(data2[1]):
                print(f"{index} -> {index2}")
                # insert_group(index, index2)

1 -> 2
4 -> 5
4 -> 6
5 -> 4
5 -> 6
6 -> 4
6 -> 5


In [9]:
# dump_algorithms([x for (x,_) in res4], "./exp_out/diff4_code.py", for_ibm=True)
# dump_algorithms([x for (x,_) in res5], "./exp_out/diff5_code.py", for_ibm=True)
# dump_algorithms([x for (x,_) in res6], "./exp_out/diff6_code.py", for_ibm=True)

In [10]:
def get_group(backend, bi, algorithms, horizon=5):
    for (index, data) in enumerate(algorithms):
        for (b, emb_i) in data[1]:
            if b == backend and emb_i == bi:
                if index == 1:
                    return 2
                elif horizon == 5:
                    if index == 6:
                        return 0
                    elif index == 3:
                        return 5
                    elif index == 4:
                        return 2
                elif horizon == 6:
                    if index in [4,5,6]:
                        return 6
                    elif index ==8:
                        return 2
                    elif index == 3:
                        return 7
                else:
                    assert False
                return index

    assert False

In [11]:
Precision.PRECISION = 5
Precision.update_threshold()

backends = []
emb_indices = []
prob0_errors = []
prob1_errors = []
diff_ = []
horizon5_group = []
horizon6_group = []
for (backend, embeddings) in backends_w_embs:
    noise_model = get_ibm_noise_model(backend)
    for (index, embedding) in enumerate(embeddings):
        target = embedding[2]
        noise_data = noise_model.get_qubit_readout_error(target)
        
        assert len(noise_data) == 1
        probs = noise_data[0].probabilities
        success0 = probs[0][1]
        success1 = probs[1][1]
        backends.append(f"{backend}{index}".replace("fake_", ""))
        emb_indices.append(index)
        prob0_errors.append(round(success0, 5))
        prob1_errors.append(round(success1, 5))
        diff_.append(round(success0 - success1, 5))
        horizon5_group.append(get_group(backend, index, res5, horizon=5))
        horizon6_group.append(get_group(backend, index, res6, horizon=6))
df = pd.DataFrame.from_dict({
    'backend': backends,
    'embedding_index': emb_indices,
    'success0': prob0_errors,
    'success1': prob1_errors,
    'success0-success1': diff_,
    'group5': horizon5_group,
    'group6': horizon6_group
})

In [12]:
df[['group5', 'success0', 'success1', 'success0-success1']].groupby(['group5']).mean()

,success0,success1,success0-success1
group5,,,
0,0.802793,0.902582,-0.099789
2,0.955320,0.912147,0.043173
5,0.961067,0.961067,0.000000


In [13]:
df.groupby(['group5', 'backend']).mean()

embedding_index  success0  success1  success0-success1  \
group5 backend                                                                 
0      cairo1                     1.0   0.97580   0.99060           -0.01480   
       cairo3                     3.0   0.94700   0.95960           -0.01260   
       cusco0                     0.0   0.28667   0.97333           -0.68667   
       cusco2                     2.0   0.61400   0.67960           -0.06560   
       cusco3                     3.0   0.68080   0.72360           -0.04280   
       cusco5                     5.0   0.78940   0.90280           -0.11340   
       cusco6                     6.0   0.99580   0.99720           -0.00140   
       hanoi0                     0.0   0.95720   0.97720           -0.02000   
       johannesburg1              1.0   0.70820   0.94640           -0.23820   
       mumbai0                    0.0   0.89780   0.89980           -0.00200   
       perth1                     1.0   0.98710   0.98710            0.00000   
       tenerife1                  1.0   0.79375   0.79375            0.00000   
2      cairo0                     0.0   0.89920   0.88800            0.01120   
       cairo2                     2.0   0.99580   0.99280            0.00300   
       cusco1                     1.0   0.64660   0.61940            0.02720   
       cusco4                     4.0   0.95500   0.51220            0.44280   
       hanoi1                     1.0   0.93060   0.91300            0.01760   
       hanoi2                     2.0   0.99580   0.99160            0.00420   
       johannesburg0              0.0   0.98460   0.96240            0.02220   
       lagos0                     0.0   0.99180   0.99040            0.00140   
       lagos1                     1.0   0.99720   0.99420            0.00300   
       lagos2                     2.0   0.99360   0.98640            0.00720   
       mumbai1                    1.0   0.99020   0.98160            0.00860   
       mumbai2                    2.0   0.99440   0.97540            0.01900   
       nairobi0                   0.0   0.98720   0.95800            0.02920   
       nairobi1                   1.0   0.98580   0.95540            0.03040   
       nairobi2                   2.0   0.98200   0.96140            0.02060   
5      perth0                     0.0   0.98260   0.98260            0.00000   
       perth2                     2.0   0.97960   0.97960            0.00000   
       tenerife0                  0.0   0.92100   0.92100            0.00000   

                      group6  
group5 backend                
0      cairo1            2.0  
       cairo3            2.0  
       cusco0            0.0  
       cusco2            2.0  
       cusco3            2.0  
       cusco5            2.0  
       cusco6            2.0  
       hanoi0            2.0  
       johannesburg1     0.0  
       mumbai0           2.0  
       perth1            2.0  
       tenerife1         2.0  
2      cairo0            2.0  
       cairo2            2.0  
       cusco1            2.0  
       cusco4            6.0  
       hanoi1            2.0  
       hanoi2            2.0  
       johannesburg0     2.0  
       lagos0            7.0  
       lagos1            7.0  
       lagos2            2.0  
       mumbai1           7.0  
       mumbai2           2.0  
       nairobi0          7.0  
       nairobi1          2.0  
       nairobi2          2.0  
5      perth0            7.0  
       perth2            2.0  
       tenerife0         2.0

In [14]:
df[['group6', 'success0', 'success1', 'success0-success1']].groupby(['group6']).mean()

,success0,success1,success0-success1
group6,,,
0,0.497435,0.959865,-0.462435
2,0.906711,0.911711,-0.005000
6,0.955000,0.512200,0.442800
7,0.989800,0.981360,0.008440


In [15]:
df.groupby(['group6', 'backend']).mean()

embedding_index  success0  success1  success0-success1  \
group6 backend                                                                 
0      cusco0                     0.0   0.28667   0.97333           -0.68667   
       johannesburg1              1.0   0.70820   0.94640           -0.23820   
2      cairo0                     0.0   0.89920   0.88800            0.01120   
       cairo1                     1.0   0.97580   0.99060           -0.01480   
       cairo2                     2.0   0.99580   0.99280            0.00300   
       cairo3                     3.0   0.94700   0.95960           -0.01260   
       cusco1                     1.0   0.64660   0.61940            0.02720   
       cusco2                     2.0   0.61400   0.67960           -0.06560   
       cusco3                     3.0   0.68080   0.72360           -0.04280   
       cusco5                     5.0   0.78940   0.90280           -0.11340   
       cusco6                     6.0   0.99580   0.99720           -0.00140   
       hanoi0                     0.0   0.95720   0.97720           -0.02000   
       hanoi1                     1.0   0.93060   0.91300            0.01760   
       hanoi2                     2.0   0.99580   0.99160            0.00420   
       johannesburg0              0.0   0.98460   0.96240            0.02220   
       lagos2                     2.0   0.99360   0.98640            0.00720   
       mumbai0                    0.0   0.89780   0.89980           -0.00200   
       mumbai2                    2.0   0.99440   0.97540            0.01900   
       nairobi1                   1.0   0.98580   0.95540            0.03040   
       nairobi2                   2.0   0.98200   0.96140            0.02060   
       perth1                     1.0   0.98710   0.98710            0.00000   
       perth2                     2.0   0.97960   0.97960            0.00000   
       tenerife0                  0.0   0.92100   0.92100            0.00000   
       tenerife1                  1.0   0.79375   0.79375            0.00000   
6      cusco4                     4.0   0.95500   0.51220            0.44280   
7      lagos0                     0.0   0.99180   0.99040            0.00140   
       lagos1                     1.0   0.99720   0.99420            0.00300   
       mumbai1                    1.0   0.99020   0.98160            0.00860   
       nairobi0                   0.0   0.98720   0.95800            0.02920   
       perth0                     0.0   0.98260   0.98260            0.00000   

                      group5  
group6 backend                
0      cusco0            0.0  
       johannesburg1     0.0  
2      cairo0            2.0  
       cairo1            0.0  
       cairo2            2.0  
       cairo3            0.0  
       cusco1            2.0  
       cusco2            0.0  
       cusco3            0.0  
       cusco5            0.0  
       cusco6            0.0  
       hanoi0            0.0  
       hanoi1            2.0  
       hanoi2            2.0  
       johannesburg0     2.0  
       lagos2            2.0  
       mumbai0           0.0  
       mumbai2           2.0  
       nairobi1          2.0  
       nairobi2          2.0  
       perth1            0.0  
       perth2            5.0  
       tenerife0         5.0  
       tenerife1         0.0  
6      cusco4            2.0  
7      lagos0            2.0  
       lagos1            2.0  
       mumbai1           2.0  
       nairobi0          2.0  
       perth0            5.0

In [16]:
df_simulator = pd.read_csv("./exp_out/ibm_simulators.csv")

In [17]:
df_simulator[(df_simulator.real_hardware == 'fake_cairo0') & (df_simulator.horizon == 5)]

,diff_index,horizon,real_hardware,acc,ins
12,0,5,fake_cairo0,0.878,-1
35,1,5,fake_cairo0,0.891,-1
58,2,5,fake_cairo0,0.888,-1
81,3,5,fake_cairo0,0.880,-1
104,4,5,fake_cairo0,0.887,-1
127,5,5,fake_cairo0,0.879,-1
150,6,5,fake_cairo0,0.885,-1


# more insights into algorithms

In [18]:
redundant_programs5 = [1,6, 5]

In [19]:
from utils import *
from bit_flip_experiments import get_initial_states, is_target_qs
from game import get_real_accuracy

In [65]:
Precision.PRECISION = 5
Precision.update_threshold()
alg1 = 4
alg2 = 2

In [66]:
res5[4]

(<utils.AlgorithmNode at 0x17f914ee0>,
 {('fake_hanoi', 1),
  ('fake_lagos', 1),
  ('fake_lagos', 2),
  ('fake_mumbai', 2),
  ('fake_nairobi', 0)})

In [85]:
m = {0: 7, 1: 1, 2: 4}
backend = FAKE_MUMBAI

In [91]:
perth_model = get_ibm_noise_model(backend, path_prefix="../")

In [98]:
perth_model.get_coupler_noise_obj(1,2)[0].apply_instructions

[[Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.Z,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.Z,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.Z,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Instruction.Z,
  Instruction.I,
  Instruction.X,
  Instruction.Z,
  Instruction.Y],
 [Instruction.I,
  Instruction.I,
  Inst

In [87]:
get_real_accuracy(res5[alg1][0], backend, get_initial_states(m), is_target_qs, m, 5, precision=5)

0.98044

In [88]:
get_real_accuracy(res5[alg2][0], backend, get_initial_states(m), is_target_qs, m, 5, precision=5)

0.98048

In [89]:
get_real_accuracy(res5[alg1][0], backend, get_initial_states(m), is_target_qs, m, 5, precision=6)

0.981168

In [90]:
get_real_accuracy(res5[alg2][0], backend, get_initial_states(m), is_target_qs, m, 5, precision=6)

0.981178